# سیشن 6 – کثیر مرحلہ پائپ لائن (منصوبہ → عملدرآمد → بہتر بنائیں)

راؤٹنگ فنکشنز کا استعمال کرتے ہوئے ہر مرحلے کے لیے ماڈلز کا انتخاب کرتا ہے، پھر حتمی جواب کو بہتر بناتا ہے۔


### وضاحت: ڈیپینڈنسی انسٹالیشن
`foundry-local-sdk` اور `openai` انسٹال کرتا ہے جو ہر قدم کے چیٹ کالز کے لیے ضروری ہیں۔ دوبارہ چلانا محفوظ ہے۔


# منظر نامہ
کثیر مرحلہ ماڈل پائپ لائن جو: (1) کسی کام کو واضح مراحل میں منصوبہ بندی کرتی ہے، (2) ہر مرحلے کو ارادے پر مبنی ماڈل کے انتخاب کے ساتھ انجام دیتی ہے، (3) ایک حتمی مرکب جواب کو بہتر بناتی ہے۔ مختلف SLM صلاحیتوں کو جوڑنے کی کارکردگی کو ظاہر کرتی ہے جبکہ معیار کو برقرار رکھتی ہے۔


In [13]:
!pip install -q foundry-local-sdk openai

### وضاحت: بنیادی درآمدات  
ارادے کی شناخت کے لیے regex درآمد کرتا ہے، فی-عرفی نام منسلک کے لیے Foundry Local manager، اور چیٹ مکمل کرنے کے لیے OpenAI client۔  


In [14]:
import re
from foundry_local import FoundryLocalManager
from openai import OpenAI

### وضاحت: قابلیت کیٹلاگ اور قواعد  
ایک قابلیت سے آگاہ کیٹلاگ اور ریجیکس قواعد کی وضاحت کرتا ہے جو قدم کے متن کو ارادے کی اقسام (کوڈ، خلاصہ، درجہ بندی، عمومی) سے جوڑنے کے لیے استعمال ہوتے ہیں۔ جب متعدد ماڈلز کسی ارادے کی حمایت کرتے ہیں تو کم ترجیحی اقدار ٹائی میں جیتتی ہیں۔


In [15]:
CATALOG = {
 'phi-4-mini': {'capabilities':['general','summarize'],'priority':2},
 'qwen2.5-coder-7b': {'capabilities':['code','refactor'],'priority':1},
 'qwen2.5-0.5b': {'capabilities':['classification','fast'],'priority':3},
}
RULES = [
 (re.compile('code|refactor|function', re.I), 'code'),
 (re.compile('summari|abstract|tl;dr', re.I), 'summarize'),
 (re.compile('classif|category|label', re.I), 'classification'),
]

### وضاحت: ارادہ، ماڈل کا انتخاب اور چیٹ مددگار
فراہم کرتا ہے:
- `detect_intent` ریگیکس پر مبنی درجہ بندی کے لیے۔
- `pick_model` صلاحیت اور ترجیح کے لحاظ سے بہترین عرف منتخب کرنے کے لیے۔
- `chat` سہولت فراہم کرنے والا جو ہر عرف کے لیے کلائنٹ بناتا ہے اور ایک بار کا جواب واپس کرتا ہے۔


In [16]:
def detect_intent(text: str):
    """Return an intent label based on simple regex rules; falls back to 'general'."""
    for pat, intent in RULES:
        if pat.search(text):
            return intent
    return 'general'

def pick_model(intent: str) -> str:
    """Pick the best model for an intent using capability match first, then priority."""
    ranked = []
    for name, meta in CATALOG.items():
        ranked.append((name, intent in meta['capabilities'], meta['priority']))
    # Sort: capability match (True first), then lower priority value
    ranked.sort(key=lambda t: (not t[1], t[2]))
    return ranked[0][0]

def chat(alias: str, content: str, temp: float = 0.4) -> str:
    """Simple helper to send a single user message to a Foundry Local model via OpenAI client."""
    m = FoundryLocalManager(alias)
    client = OpenAI(base_url=m.endpoint, api_key=m.api_key or 'not-needed')
    mid = m.get_model_info(alias).id
    resp = client.chat.completions.create(
        model=mid,
        messages=[{'role': 'user', 'content': content}],
        max_tokens=180,
        temperature=temp,
    )
    return resp.choices[0].message.content

### وضاحت: ملٹی-اسٹیپ پائپ لائن فنکشن
پائپ لائن کو نافذ کرتا ہے: منصوبہ بندی → مراحل کو تجزیہ کریں → ہر ایک کو ارادے پر مبنی روٹنگ کے ساتھ انجام دیں → مشترکہ نتائج کو بہتر بنائیں۔ معائنہ یا تشخیص کے لیے منظم ڈکشنری واپس کرتا ہے۔


In [17]:
def pipeline(task: str):
    """Multi-step pipeline: plan → execute steps → refine final answer.

    Returns dict with keys: plan (raw plan text), steps (list of tuples), final (refined answer).
    Each step tuple: (index, step_text, step_result, model_alias_used)
    """
    # 1. Plan
    plan_alias = pick_model('general')
    plan_prompt = (
        "Break the task into 3 concise, actionable steps (no extra commentary).\n"
        f"Task: {task}"
    )
    plan = chat(plan_alias, plan_prompt)

    # 2. Parse steps (robust to numbering or bullet styles)
    raw_lines = [l.strip() for l in plan.splitlines() if l.strip()]
    steps = []
    for line in raw_lines:
        cleaned = re.sub(r'^\d+[).:-]?\s*', '', line)  # remove leading numbering
        cleaned = re.sub(r'^[-*]\s*', '', cleaned)      # remove bullet markers
        if cleaned:
            steps.append(cleaned)
        if len(steps) == 3:
            break
    if not steps:
        steps = [task]

    # 3. Execute steps
    outputs = []
    for idx, step in enumerate(steps, 1):
        intent = detect_intent(step)
        exec_alias = pick_model(intent)
        exec_prompt = (
            f"Execute step {idx} for the overall task.\n"
            f"Overall task: {task}\n"
            f"Step {idx}: {step}\n"
            "Return a concise result focusing only on the step objective."
        )
        result = chat(exec_alias, exec_prompt)
        outputs.append((idx, step, result, exec_alias))

    # 4. Refine final answer
    refine_alias = pick_model('summarize')
    combined = "\n\n".join(
        f"Step {idx} ({alias}) Output:\n{res}" for idx, _step, res, alias in outputs
    )
    refine_prompt = (
        "You are a senior assistant. Synthesize these step outputs into a cohesive final answer.\n"
        "Ensure clarity, avoid repetition, and highlight improvements or key insights if relevant.\n\n"
        f"Task: {task}\n\n"
        f"Step Outputs:\n{combined}"
    )
    final_ans = chat(refine_alias, refine_prompt)

    return {"plan": plan, "steps": outputs, "final": final_ans}

### وضاحت: مثال کے طور پر کام چلائیں
پورے پائپ لائن کو ایک ریفیکٹر پر مبنی کام پر چلانے کی وضاحت کرتا ہے جو مخلوط ارادوں (کوڈ + خلاصہ) کو ظاہر کرتا ہے۔ نتیجہ ڈکشنری خام منصوبہ، ہر مرحلے کے نتائج منتخب کردہ ماڈل کے ناموں کے ساتھ، اور آخری مرتب شدہ جواب دکھاتی ہے۔


In [18]:
result = pipeline('Generate a refactored version of a slow Python loop and summarize performance gains.')
result

{'plan': '1. Profile the existing slow Python loop to identify bottlenecks.\n2. Refactor the loop using optimized techniques (e.g., list comprehensions, map, or itertools).\n3. Compare the performance of the refactored loop with the original using a benchmarking tool and summarize the gains.',
 'steps': [(1,
   'Profile the existing slow Python loop to identify bottlenecks.',
   "To execute step 1, you would use a profiling tool like `cProfile` in Python to analyze the performance of the existing slow loop. Here's an example of how you might do this:\n\n```python\nimport cProfile\n\ndef slow_loop():\n    # Example of a slow loop\n    result = []\n    for i in range(1000000):\n        result.append(i * i)\n    return result\n\n# Profile the slow loop\ncProfile.run('slow_loop()', 'profile_stats')\n\n# To see the results, you can use pstats module\nimport pstats\np = pstats.Stats('profile_stats')\np.sort_stats('cumulative').print_stats()\n```\n\nThis code will run the `slow_loop` function

### وضاحت: نتیجہ آبجیکٹ دکھائیں  
منظم پائپ لائن آؤٹ پٹ کو فوری معائنہ یا نیچے کی سطح پر جائزہ لینے کے لیے دکھاتا ہے (مثلاً، مرحلے کے معیار یا بہتری کی مؤثریت کو جانچنے کے لیے)۔



---

**ڈسکلیمر**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کا استعمال کرتے ہوئے ترجمہ کی گئی ہے۔ ہم درستگی کے لیے کوشش کرتے ہیں، لیکن براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا غیر درستیاں ہو سکتی ہیں۔ اصل دستاویز کو اس کی اصل زبان میں مستند ذریعہ سمجھا جانا چاہیے۔ اہم معلومات کے لیے، پیشہ ور انسانی ترجمہ کی سفارش کی جاتی ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کے لیے ہم ذمہ دار نہیں ہیں۔
